To improve clarity and maintainability, the Goodreads dataset analysis is
divided into multiple notebooks, each dedicated to a single phase of the
data science workflow. This notebook covers the current stage only.



**Navigation**
- ▶️ [01. Data Cleaning](https://www.kaggle.com/code/madisrinija/goodreads-dataset-cleaning)
- 📊 [02. Exploratory Data Analysis](https://www.kaggle.com/code/madisrinija/goodreads-dataset-eda/)
- 🧠 [03. Feature Engineering](03_feature_engineering.ipynb)
- 🤖 [04. Modeling](04_modeling.ipynb)

# PART 1: DATA CLEANING

In [ ]:
import numpy as np
import pandas as pd
import os
import sqlite3
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
import string

##  STEP 0: Import & Inspect

Start by loading both datasets and seeing their structure.

In [ ]:
details = pd.read_csv("/kaggle/input/books-dataset-goodreadsmay-2024/Book_Details.csv")
DB_PATH = "/kaggle/input/books-dataset-goodreadsmay-2024/book_reviews.db"

conn = sqlite3.connect(DB_PATH)
reviews = pd.read_sql_query(
    "SELECT * FROM book_reviews", conn
)
conn.close()

In [ ]:
#details = details.drop(columns=["Unnamed: 0","cover_image_uri","authorlink",])

In [ ]:
details

In [ ]:
reviews

## STEP 1: Clean the details table

The details table is mostly metadata, not text-heavy — but it contains several stringified lists and dicts.
You’ll want to normalize them.

### 1.1 Handle stringified lists

#### Processing Genres

In [ ]:
import ast

def safe_parse(x):
    if isinstance(x, str):  
        try:
            return ast.literal_eval(x)
        except:
            return []  
    return x 

In [ ]:
details['genres'] = details['genres'].apply(safe_parse)

#### Processing Format Column

In [ ]:
def extract_format_info(text):
    if not isinstance(text, str):
        return None, None
    
    # Match number + pages + format
    match = re.match(r"(\d+)\s*pages?,\s*(.*)", text.strip())
    if match:
        pages = int(match.group(1))
        fmt = match.group(2).strip()
        return pages, fmt
    
    # Fallback if no page number present
    return None, text.strip()

In [ ]:
details['format'] = details['format'].apply(safe_parse)
details['format'] = details['format'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)

details[['pages', 'book_format']] = details['format'].apply(
    lambda x: pd.Series(extract_format_info(x))
)


#### Processing Pubishing Info

In [ ]:
def extract_date(date):
  if not isinstance(date,str):
    return None
  match = re.search(r'([A-Za-z]+ \d{1,2}, \d{4})',date)
  if match:
    return match.group(1)
  return None

In [ ]:
details['publication_info'] = details['publication_info'].apply(safe_parse)
details['publication_info'] = details['publication_info'].apply(lambda x: x[0] if isinstance(x,list) and len(x)>0 else None)
details['publication_info'] = details['publication_info'].apply(extract_date)
details['publication_date'] = pd.to_datetime(details['publication_info'], errors='coerce')

#### Processing Pages Column

In [ ]:
details['num_pages'] = details['num_pages'].apply(safe_parse)
details['num_pages'] = details['num_pages'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)
details['num_pages'] = pd.to_numeric(details['num_pages'], errors='coerce')

In [ ]:
def parse_dist(text):
  try:
    return {k:int(v.replace(',', '')) for k, v in text.items()}
  except:
    return None

In [ ]:
details['rating_distribution'] = details['rating_distribution'].apply(safe_parse)
details['rating_distribution'] = details['rating_distribution'].apply(parse_dist)

ratings_df = details['rating_distribution'].apply(pd.Series).fillna(0).astype(int)
ratings_df = ratings_df.add_prefix('rating_')
details = pd.concat([details, ratings_df], axis=1)

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
stop = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [ ]:
def clean_text(text):

    text = re.sub(r'http\S+|www\S+', '', text)            # remove URLs
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\d+', '', text)                       # remove digits
    words = [lemmatizer.lemmatize(w) for w in text.split() if w not in stop]
    return " ".join(words)

In [ ]:
reviews['review_content'] =  reviews['review_content'].apply(lambda x: x.replace('"', '')).apply(clean_text)

reviews = reviews[reviews['review_content'].notna()]

In [ ]:
def convert_followers(text):
  if not isinstance(text, str):
    return None
  text = text.replace(' follower','')
  text = text.replace(',','')
  if text.endswith('s'):
      text = text[:-1]
  if 'k' in text:
    return int(float(text.replace('k',''))*1000)
  elif 'm' in text:
    return int(float(text.replace('m',''))*1000000)
  try:
    return int(text)
  except ValueError:
    return None

reviews['reviewer_followers'] = reviews['reviewer_followers'].apply(convert_followers)

In [ ]:
def convert_total_reviews(text):
  if not isinstance(text, str):
    return None
  text = text.replace(' review','')
  text = text.replace(',','')
  if text.endswith('s'):
      text = text[:-1]

  if 'k' in text:
    return int(float(text.replace('k',''))*1000)
  elif 'm' in text:
    return int(float(text.replace('m',''))*1000000)
  try:
    return int(text)
  except ValueError:
    return None


reviews['reviewer_total_reviews'] = reviews['reviewer_total_reviews'].apply(convert_total_reviews)

In [ ]:
reviews['review_date'] = reviews['review_date'].apply(lambda x: pd.to_datetime(x, errors='coerce'))

In [ ]:
def convert_likes_on_review(text):
  if not isinstance(text, str):
    return None
  #text = text.replace(' likes','')
  #text = text.replace(',','')
  text = text.split()[0]  


  if 'k' in text:
    return int(float(text.replace('k',''))*1000)
  elif 'm' in text:
    return int(float(text.replace('m',''))*1000000)
  try:
    return int(text)
  except ValueError:
    return None


reviews['likes_on_review'] = reviews['likes_on_review'].apply(convert_likes_on_review)

In [ ]:
def extract_rating(st):
  if pd.isna(st):
    return np.nan
  lis = st.split()
  return int(lis[1])


reviews['review_rating'] = reviews['review_rating'].apply(extract_rating)

In [ ]:
details.to_csv('details.csv', index=False)
reviews.to_csv('reviews.csv',index=False)